# Applying Novel Approaches to Predict Yelp Ratings

## Synopsis

In our research we explore two novel methods for predicting user ratings on the domain of Yelp businesses. We provide a prospective business advertiser/marketer/researcher a detailed comparison of the differences between methods and suggest practical considerations for their usage.

Keywords: Machine Learning; Collaborative Filtering; Ensemble Models; Factorization Machines

#### Contributors:
- EK Itoku | UNI: ii2155 | Email: ii2155@columbia.edu
- Jason Kuo | UNI: jk4097 | Email: jk4097@columbia.edu
- Sean Xu | UNI: cx2118 | Email: cx2118@columbia.edu


#### Report Layout:

* Objectives
* Recommendation Approaches
* Evaluation Metrics
* Analysis of Results
    + Comparison of Training Data Size
    + Comparison of Hyperparameter Tuning
* Implementation Considerations
* Conclusions and Takeaways
* References

In [ ]:
# %load_ext nb_black

In [1]:
import os
import numpy as np
import pickle
import pandas as pd
from source.utils import plot_lines, create_quantile_bucket, rmse
from sklearn.metrics import mean_absolute_error as mae
import matplotlib.pyplot as plt

pd.options.display.float_format = "{:4.2f}".format
%matplotlib inline

# Constants
DEFAULT_SAMPLE_SIZE = 50  # 50,000 users
QUANTILES = 10

In [57]:
cols = [
    "user_id",
    "business_id",
    "review_stars",
    "user_average_stars",
    "user_review_count",
    "user_elite",
    "user_fans",
    "business_stars",
    "business_review_count",
]
truth = pd.read_csv("data/feature.csv", usecols=cols)

In [72]:
# [fn for fn in os.listdir("result") if fn.endswith("_pred.pkl")]

In [73]:
model_result_files = [
    "baseline_pred.pkl",
    "knnwithmeans_pred.pkl",
    "svd_pred.pkl",
    "coclustering_pred.pkl",
    "knnbasic_pred.pkl",
    "ensemble_pred.pkl",
    "fm_pred.pkl",
]
preds = []

for fn in model_result_files:
    pred = pd.read_pickle(f"result/{fn}")
    pred = pred.merge(truth, on=["user_id", "business_id"], how="inner")

    pred["squared_error"] = (pred.review_stars - pred.y_pred) ** 2
    pred["absolute_error"] = (pred.review_stars - pred.y_pred).abs()

    preds.append([fn.split("_")[0], pred])

## Objectives

In this study we aim to predict the rating of the last business a Yelp user visits. A rating is on the scale of 1 - 5 with 5 representing great. For marketing companies, this can be invaluable in identifying potential users who will like a targeted business and only focus advertising to those users with high predicted ratings.

## Evaluation Metrics

To achieve our objective, we used two different evaluation schemes:


#### - Root Mean Squared Error (Primary Benchmark)

We use the root mean squared error benchmark (RMSE) to measure the deviation of our predicted rating from the true rating in our testing set. A lower result is better signaling less prediction error.


#### - Mean Absolute Error (Secondary Benchmark)

RMSE penalizes the larger deviations from the true rating more than similarly spaced apart smaller deviations when the total amount of deviation is the same. Mean Absolute Error (MAE) measures all deviations similarly. 

## Recommendation Approaches

### 1) Baseline model (Biased Model)
<img src="image/baseline1.png" alt="drawing" width="200"/>
Where we adjust the mean rating of all users and all items by the observed deviations of user u and item i.

In order to avoid overfitting, we add regularizations by penalizing the magnitudes of the parameters.
<img src="image/baseline2.png" alt="drawing" width="400"/>

In Surprise implementation, for easier calculation, it decouples the calculation of b_u and b_i. For b_i
<img src="image/baseline3.png" alt="drawing" width="300"/>

For b_u
<img src="image/baseline4.png" alt="drawing" width="300"/>

And Surprise provides two algorithms to calculate the parameters: Baselines can be estimated in two different ways:
Stochastic Gradient Descent (SGD)
Alternating Least Squares (ALS)

We run a GridSearch for both algorithms and the accuracy matrix we use is:
- Primary: RMSE
- Secondary: MAE

Results:



Algorithm | Model with least RMSE | RMSE | Model with least MAE | MAE
----------|-----------------------|------|----------------------|----
ALS|'n_epochs': 30, 'reg_u': 10, 'reg_i': 10|1.15|'n_epochs': 30, 'reg_u': 10, 'reg_i': 10 |0.92
SGD|'n_epochs': 30, 'reg': 0.1, 'learning_rate': 0.005|1.15|'n_epochs': 30, 'reg': 0.01, 'learning_rate': 0.01|0.90


### 2) Individual and Ensemble Methods

#### Individual models
We considered several models individually and then ensembling them including:
- KNN
- KNN(Basic and With_Means)
- SVD
- baselineonly
- co-colustering

#### - KNN
The idea is to use similar users or items to predict the rating of given user and item. 

<img src="image/knn1.png" alt="drawing" width="300"/>
<img src="image/knn2.png" alt="drawing" width="300"/>

There are several ways to calculate the similarities including cosine, pearson and msd.
Results:


Algorithm|Model with least RMSE|RMSE|Model with least MAE|MAE
---------|---------------------|----|--------------------|---
KNNBasic|‘similarity_method’: cosine, 'k': 40|1.50|‘similarity_method’: msd, 'k': 20|1.12
KNNwithMean|‘similarity_method’: cosine, 'k': 40|1.39|‘similarity_method’: cosine, 'k': 40|1.05

#### - SVD
A matrix factorization method which is popularized by Simon Funk during the Netflix Prize. When baselines are not used, this is equivalent to Probabilistic Matrix Factorization. 

Algorithm|Model with least RMSE|RMSE|Model with least MAE|MAE
---------|---------------------|----|--------------------|---
SVD|'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.1|1.13|'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.1|0.89


#### - Co-Clustering

Cluster assignments are done by basic techniques like kmeans.

<img src="image/cocluster.png" alt="drawing" width="600"/>

#### Ensemble

#### - Basic Ensemble
The idea of ensemble model is to stack a set of basic recommendation models and use the weighted sum of the predictions as the new prediction which will  remove the flaws in individual implementation.
- Averaging
- Weighted Average (Train weights with linear regression and ElasticNet regression)

Steps:
1. Tune the hyperparameter of base models including:
2. Train base models using the tuned hyperparameters
3. Run regression to learn the weights or simply user average
4. Stack models together with weights (average or trained weights)

Results:


Model | RMSE
------|-----
Base Models|1.13 - 1.33
Ensemble with average weights|1.32
Ensemble with weights learned from linear regression|1.32
Ensemble with weights learned from ElasticNet|1.39

Model | MAE
------|----
Base Models|0.9 - 1
Ensemble with average weights|1.05
Ensemble with weights learned from linear regression|1.03
Ensemble with weights learned from ElasticNet|1.11


#### - Advanced Ensemble Techniques
- Stacking

### 3) Factorization Machines

Factorization Machines are a state of the art solution to recommendation problems. In theory, FMs should produce better results than MF due to interaction terms but in practice it might not necessary be the case. If side information is omitted, then FMs become the same as the MF model.

FMs are similar to collective approaches allowing for multiple relationships between entities but differ in:
- FMs use a clever feature representation to cast factorization as a regression, classification, or ranking problem
- In addition to relations between entities, FMs allow for interaction terms for items within a single entity type
- FMs can be defined such that they act like, or mimic, other techniques like SVD++





## Analysis of Results


### Model Accuracy

#### Baseline (Most Popular 5 Movies)

TODO: Update for yelp project

In our baseline method we achieved a RMSE of 1.006 and precision of 0.0337 on the test data.

#### Neighborhood-Based

In [ ]:
# with open(path_nb_result, 'rb') as f:
#     nb_result = pickle.load(f)

# test_acc = nb_result.loc[nb_result.sample_size == DEFAULT_SAMPLE_SIZE, ['rmse_test', 'top_k_precision_test']].values[0]

# print("In our neighborhood-based method we achieved a RMSE of {0} and precision of {1} on the test data."
#       .format(round(test_acc[0],4), round(test_acc[1],4)))

#### Model-Based

In [ ]:
# with open(path_als_result, 'rb') as f:
#     als_result = pickle.load(f)
    
# test_acc = als_result.loc[als_result.sample_size == DEFAULT_SAMPLE_SIZE, ['rmse_test', 'top_k_precision_test']].values[0]

# print("In our model-based method we achieved a RMSE of {0} and precision of {1} on the test data."
#       .format(round(test_acc[0],4), round(test_acc[1],4)))

TODO: Update for yelp project

Surprisingly, our baseline popularity method did resonably well in test RMSE however it predictably did poor on the precision side. The neighborhood model improved RMSE and notably precision vs. the baseline. Finally, our model-based method performed the best with RMSE and slightly worse on precision vs. the NB method on the test dataset. 

### Do your models work equally well for all users?

We will evaluate models from two different angles
##### 1. In terms of number of users
The more users review restaurants, the better our predictions perform. This is intuitive as there are more samples models to learn, we can train models better.


##### 2. In terms of number of elite status
Our experiments show that our models perform better for elite users. For example, MAE is 0.78 for elite users whereas 1.03 for non-elite users. We think this is because there are more samples for elite users. Another possibility is that  there might be some spam users or malicious business accounts that rate restaurants in an unusual manner.

In [70]:
# 1. In terms of number of users
for nm, pred in preds:
    pred["user_review_count_bin"] = pd.qcut(pred.user_review_count, 10)
    desc = pred.groupby("user_review_count_bin").absolute_error.describe()
    print(nm)
    print(desc)
    print()
    print()

baseline
                         count  mean  std  min  25%  50%  75%  max
user_review_count_bin                                             
(4.999, 7.0]          35329.00  1.12 0.81 0.00 0.51 0.95 1.59 4.00
(7.0, 9.0]            24570.00  1.10 0.82 0.00 0.46 0.92 1.56 4.00
(9.0, 11.0]           20224.00  1.07 0.80 0.00 0.45 0.90 1.52 4.00
(11.0, 14.0]          23186.00  1.04 0.79 0.00 0.43 0.87 1.49 4.00
(14.0, 18.0]          22003.00  1.02 0.79 0.00 0.42 0.86 1.45 4.00
(18.0, 25.0]          24238.00  1.01 0.79 0.00 0.39 0.84 1.45 4.00
(25.0, 38.0]          23846.00  0.98 0.78 0.00 0.38 0.81 1.40 4.00
(38.0, 68.0]          24415.00  0.93 0.75 0.00 0.36 0.77 1.31 4.00
(68.0, 161.0]         24665.00  0.88 0.70 0.00 0.35 0.72 1.23 4.00
(161.0, 13278.0]      24647.00  0.79 0.63 0.00 0.31 0.66 1.10 4.00


knnwithmeans
                         count  mean  std  min  25%  50%  75%  max
user_review_count_bin                                             
(4.999, 7.0]          35329.00  1.18 1

In [71]:
# 2. In terms of number of elite status

for nm, pred in preds:
    pred.user_elite = pred.user_elite > 0
    desc = pred.groupby("user_elite").absolute_error.describe()
    print(nm)
    print(desc)
    print()
    print()

baseline
               count  mean  std  min  25%  50%  75%  max
user_elite                                              
False      213281.00  1.03 0.79 0.00 0.42 0.86 1.47 4.00
True        33842.00  0.78 0.63 0.00 0.31 0.64 1.09 4.00


knnwithmeans
               count  mean  std  min  25%  50%  75%  max
user_elite                                              
False      213281.00  1.07 0.93 0.00 0.31 0.88 1.54 4.00
True        33842.00  0.81 0.69 0.00 0.29 0.66 1.15 4.00


svd
               count  mean  std  min  25%  50%  75%  max
user_elite                                              
False      213281.00  1.04 0.77 0.00 0.46 0.87 1.44 4.00
True        33842.00  0.77 0.62 0.00 0.31 0.65 1.07 4.00


coclustering
               count  mean  std  min  25%  50%  75%  max
user_elite                                              
False      213281.00  1.07 0.97 0.00 0.26 0.88 1.57 4.00
True        33842.00  0.83 0.71 0.00 0.29 0.68 1.16 4.00


knnbasic
               count  mean  std 

### Do your models work equally well for all restaurants?

Our models perform better for the restaurants that are popular with more reviews. Similar to the user analysis above, models perform better if there are more data to learn patterns on.

In [52]:
for nm, pred in preds:
    pred["business_review_count_bin"] = pd.qcut(pred.business_review_count, 10)
    desc = pred.groupby("business_review_count_bin").absolute_error.describe()
    print(nm)
    print(desc)
    print()
    print()

baseline
                             count  mean  std  min  25%  50%  75%  max
business_review_count_bin                                             
(49.999, 84.0]            25035.00  1.10 0.80 0.00 0.47 0.96 1.59 4.00
(84.0, 125.0]             24420.00  1.05 0.78 0.00 0.44 0.89 1.50 4.00
(125.0, 173.0]            25025.00  1.06 0.78 0.00 0.44 0.90 1.52 4.00
(173.0, 236.0]            24782.00  1.03 0.78 0.00 0.43 0.86 1.47 4.00
(236.0, 313.0]            24431.00  1.02 0.79 0.00 0.41 0.84 1.45 4.00
(313.0, 413.0]            24706.00  1.00 0.78 0.00 0.41 0.83 1.40 4.00
(413.0, 555.0]            24662.00  0.98 0.77 0.00 0.39 0.80 1.37 4.00
(555.0, 851.0]            24650.00  0.97 0.77 0.00 0.38 0.78 1.35 4.00
(851.0, 1476.0]           24765.00  0.90 0.75 0.00 0.36 0.72 1.22 4.00
(1476.0, 8348.0]          24647.00  0.87 0.71 0.00 0.34 0.72 1.19 4.00


knnwithmeans
                             count  mean  std  min  25%  50%  75%  max
business_review_count_bin                            

### Think carefully about a business or technical framework to segment your data for users, items, or more, and test accuracy separately for these segments

TODO

### Comparison of Hyper Parameter Tuning

#### Neighborhood-Based: Increasing K-Neighbors

In [ ]:
# with open(path_nb_100k_hyper_rmse, 'rb') as f:
#     nb_100k_hyper_rmse = pickle.load(f)
    
# with open(path_nb_100k_hyper_precision, 'rb') as f:
#     nb_100k_hyper_precision = pickle.load(f)

# k = nb_100k_hyper_rmse[1].values
# rmse = nb_100k_hyper_rmse[2].values
# precision = nb_100k_hyper_precision[2].values
    
# plot_lines(k, rmse, 'Neighborhood-Based RMSE vs. K-Neighbors', 'K', 'RMSE')
# plot_lines(k, precision, 'Neighborhood-Based Precision vs. K-Neighbors', 'K', 'Precision')

In [ ]:
# min_rmse_idx = np.argmin(rmse)
# best_k = k[min_rmse_idx]
# lowest_RMSE = rmse[min_rmse_idx]
# print("K of {0} looks to be the result with the lowest RMSE ({1}) on the test data."
#       .format(round(best_k,4), round(lowest_RMSE,4)))

# max_precision_idx = np.argmax(precision)
# best_k = k[max_precision_idx]
# highest_precision = precision[max_precision_idx]
# print("K of {0} looks to be the result with the highest precision ({1}) on the test data."
#       .format(round(best_k,4), round(highest_precision,4)))

TODO: Update for yelp project

From the data, we can see the hyperparameter K for the number of neighbors to calculate similarity with can be tuned to improved RMSE however at a cost of precision. It appears that using a smaller clique of peers improves the likelihood of finding a desirable top 5 movie however it comes with a tradeoff of being less accurate in predicting its exact rating. Perhaps favoring precision in this tradeoff is resonable for a business to optimize if they are looking to maximize movie visits and worry less about how people thought of them.

#### Model-Based: Increasing Regularization Parameter (λ)

In [ ]:
#TODO: Update for yelp project

# with open(path_als_100k_hyper_rmse, 'rb') as f:
#     als_100k_hyper_rmse = pickle.load(f)

# lam_cv = als_100k_hyper_rmse[1].values
# rmse_cv = als_100k_hyper_rmse[2].values

# param_size = int(len(lam_cv)/CV_K_FOLDS)
# lam = lam_cv[:param_size]
# rmse_per_kfold = np.array_split(rmse_cv, CV_K_FOLDS)
# rmse = np.mean(rmse_per_kfold, axis=0)

# plot_lines(lam, rmse, 'Model-Based RMSE vs. Regularization Parameter', 'λ', 'RMSE')

In [ ]:
#TODO: Update for yelp project

# min_rmse_idx = np.argmin(rmse)
# best_lamda = lam[min_rmse_idx]
# lowest_RMSE = rmse[min_rmse_idx]
# print("Lambda of {0} looks to be the result with the lowest RMSE ({1}) on the test data."
#       .format(round(best_lamda,4), round(lowest_RMSE,4)))

TODO: Update for yelp project

As we tweaked the regularization parameter lambda, we were able to find an optimal level to minimize RMSE. This lambda is used for the Alternating Least Squares algorithm to generalize the model better through introducing penalty to latent factor weights.

In our final model we tuned additional hyperparameters for max iterations the model should run and rank (dimensionality of latent factors) through cross validation however due to model package limitations were unable to extract the outputs in an efficient manner. Similarly, the precision metric was not easily exposed on a hyperparameter tuning level of which we omit.

## Implementation Considerations

For any business looking to employ one of these methods into real-world use, they should consider the following:

#### Cold Start

TODO: Update for yelp project

In the case the business has no existing user-item recommendation data, the company can ensure a randomized selection of movie predictions in order to generate a dataset with a resonable selection of movies to recommend. As we seen in data scaling, introducing too wide of a selection of movies can make prediction accuracy suffer due to the sparcity of ratings in the neighborhood-based method but can be partially addressed in the model-based matrix factorization method.

#### New Users

TODO: Update for yelp project

With no existing data, it is question on how to provide an appropriate recommendation for new users. We suggest an employing a popularity-based approach (i.e. suggesting the top-k most popular items) as it has been shown to provide a decent performance vs. the methods incorporating additional user data. The business can continue to approach until the user has accumulated a sufficient number movie interactions at which the predictions can be reliably estimated (a business rule threshold for the number of required ratings can be implemented). In a follow-up we would like to explore this frontier of number of movie ratings vs. prediction accuracy. From this data, a business may want to employ multiple models for light-users and heavy-users of the platform using the number of ratings as a proxy.

#### Online vs. Offline

TODO: Update for yelp project

We would recommend matrix factorization for an online method as the results shows it provides good accuracy as data grows and its latent factor representation is more scalable in runtime (based on our observations of less than linear growth from 50k to 150k datapoints) and memory (unable to run past 150k datapoints using a high-end consumer laptop). Hosting the entire growing dataset to provide a neighborhood-based recommendation will see rapidly increasing resource demands.


#### Method Accuracy

TODO: Update for yelp project

A method that provides a < 20%  precision of a top 5 recommendation approach (1/5 = 20%) may be considered insufficient for the business to release as it is not able to recommend a single movie an user will watch reliably. Thus the business can establish a minimal accuracy threshold as a business rule for real-world implementation. In our experiment, the popularity based baseline would not meet this hurdle while both collaborative filtering method would pass.


#### Recommendation Diversity

TODO: Update for yelp project

Recommendation diversity, as measured by our coverage metric, can be a notable consideration. A business like Netflix could favor a high level of coverage across their catalogue to limit their showings of a smaller selection of blockbuster movies if they a higher pay-per-view licensing fee to content distributors. If they could recommend more indie or niche movies while providing high levels of user engagement then they could save on licensing costs.


## Conclusions and Takeaways

TODO: Update for yelp project

We have shown that both memory-based and model-based collaborative filtering approaches are effective at providing movie recommendations better in terms of accuracy and diversity than a baseline popularity-based approach. We provided practical implementation considerations for businesses wishing to adopt one of our approaches. We have developed a Spark ML implementation of the collaborative filtering methods that takes advantage of a distributed computing framework for easier resource scaling. The researchers would like to continue to investigate the impact of the number of user ratings on accuracy, effects of even larger datasets and taking into consideration the types of movie genres or release years the user has rated as a input to improve recommendations.






## References

1: https://towardsdatascience.com/precision-vs-recall-386cf9f89488

2: https://surprise.readthedocs.io/en/stable/index.html